In [1]:
import os
import sys
import time
import datetime
import argparse
import pickle
from skimage.data import imread
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

sys.path.append('/Users/mike/Desktop/airbus_kaggle/src/PyTorch-YOLOv3_lite')

In [2]:
%load_ext autoreload
%autoreload 2
from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

In [15]:
args = ['--image_folder','/Users/mike/Desktop/airbus_kaggle/data/train',
        '--box_file','/airbus/box_data.pickle',
        '--model_config_path','/Users/mike/Desktop/airbus_kaggle/src/PyTorch-YOLOv3_lite/config/classifier.cfg',
        '--weights_path','/Users/mike/Desktop/airbus_kaggle/classifier_weights/6.weights',
        '--checkpoint_dir','/Users/mike/Desktop/airbus_kaggle/output',
        '--img_size','64',
        '--epochs','30',
        '--n_cpu','4',
        '--use_cuda','True']
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=30, help='number of epochs')
parser.add_argument('--image_folder', type=str, default='data/samples', help='path to dataset')
parser.add_argument('--box_file', type=str, default='data/samples', help='path to box data')
parser.add_argument('--batch_size', type=int, default=16, help='size of each image batch')
parser.add_argument('--model_config_path', type=str, default='config/yolov3.cfg', help='path to model config file')
parser.add_argument('--weights_path', type=str, default='weights/yolov3.weights', help='path to weights file')
parser.add_argument('--conf_thres', type=float, default=0.8, help='object confidence threshold')
parser.add_argument('--nms_thres', type=float, default=0.4, help='iou thresshold for non-maximum suppression')
parser.add_argument('--n_cpu', type=int, default=0, help='number of cpu threads to use during batch generation')
parser.add_argument('--img_size', type=int, default=416, help='size of each image dimension')
parser.add_argument('--checkpoint_interval', type=int, default=1, help='interval between saving model weights')
parser.add_argument('--checkpoint_dir', type=str, default='checkpoints', help='directory where model checkpoints are saved')
parser.add_argument('--use_cuda', type=bool, default=True, help='whether to use cuda if available')
opt = parser.parse_args(args)
print(opt)


Namespace(batch_size=16, box_file='/airbus/box_data.pickle', checkpoint_dir='/Users/mike/Desktop/airbus_kaggle/output', checkpoint_interval=1, conf_thres=0.8, epochs=30, image_folder='/Users/mike/Desktop/airbus_kaggle/data/train', img_size=64, model_config_path='/Users/mike/Desktop/airbus_kaggle/src/PyTorch-YOLOv3_lite/config/classifier.cfg', n_cpu=4, nms_thres=0.4, use_cuda=True, weights_path='/Users/mike/Desktop/airbus_kaggle/classifier_weights/6.weights')


In [12]:
cuda = torch.cuda.is_available() and opt.use_cuda

os.makedirs(opt.checkpoint_dir, exist_ok=True)

# Get hyper parameters
hyperparams     = parse_model_config(opt.model_config_path)[0]
learning_rate   = float(hyperparams['learning_rate'])
momentum        = float(hyperparams['momentum'])
decay           = float(hyperparams['decay'])
burn_in         = int(hyperparams['burn_in'])

# Initiate model
model = Darknet_Classifier(opt.model_config_path)
model.load_weights(opt.weights_path)
model.apply(weights_init_normal)

if cuda:
    model = model.cuda()

model.train();



In [13]:
# Get dataloader
outfile = '../../data/box_data.pickle'

with open(outfile, 'rb') as f:
    data = pickle.load(f)
    
dataloader = torch.utils.data.DataLoader(
    BoxedDataset('../../data/train',(768,768),data),
    batch_size=opt.batch_size, shuffle=True, num_workers=opt.n_cpu)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, dampening=0, weight_decay=decay)


In [18]:
class BoxDetections(Dataset):
    def __init__(self,imgdir,imgsize,data):
        super().__init__()
        self.imgdir = imgdir
        self.imgsize = imgsize
        self.box_image_ids = data['box_image_ids']
        self.L1 = data['L1']
        self.L2 = data['L2']
        self.R0 = data['C0'] # accidently switched when saved
        self.C0 = data['R0'] # accidently switched when saved
        self.theta = data['theta']
        self.all_image_ids = data['all_image_ids']
        self.num_ships = data['num_ships']
        self.total_ships = sum(self.num_ships)
        self.total_imgs = len(self.all_image_ids)
    def __len__(self):
        return self.total_imgs
    def __getitem__(self,idx):
        img_path = os.path.join(self.imgdir,self.all_image_ids[idx])
        img = np.array(Image.open(img_path)) 
        # Channels-first
        img = np.transpose(img, (2, 0, 1))
        # As pytorch tensor
        img = torch.from_numpy(img).float()
        return img  
    

In [19]:
bdtest = BoxDetections(opt.image_folder,(768,768),data)
bdtest.__getitem__(0)

tensor([[[ 30.,  30.,  28.,  ...,  30.,  35.,  36.],
         [ 29.,  30.,  30.,  ...,  28.,  31.,  30.],
         [ 28.,  31.,  32.,  ...,  27.,  28.,  26.],
         ...,
         [ 25.,  27.,  24.,  ...,  20.,  26.,  28.],
         [ 25.,  25.,  23.,  ...,  19.,  25.,  25.],
         [ 21.,  21.,  20.,  ...,  28.,  30.,  27.]],

        [[ 49.,  49.,  47.,  ...,  47.,  52.,  53.],
         [ 48.,  49.,  49.,  ...,  45.,  48.,  47.],
         [ 47.,  50.,  51.,  ...,  44.,  45.,  43.],
         ...,
         [ 49.,  51.,  48.,  ...,  44.,  50.,  52.],
         [ 49.,  49.,  47.,  ...,  43.,  49.,  49.],
         [ 45.,  45.,  44.,  ...,  52.,  54.,  51.]],

        [[ 79.,  79.,  77.,  ...,  77.,  82.,  83.],
         [ 78.,  79.,  79.,  ...,  73.,  78.,  77.],
         [ 77.,  80.,  81.,  ...,  72.,  75.,  73.],
         ...,
         [ 77.,  79.,  76.,  ...,  70.,  76.,  78.],
         [ 77.,  77.,  75.,  ...,  69.,  75.,  75.],
         [ 73.,  73.,  72.,  ...,  78.,  80.,  77.]]]